# Introduction to Rebound Question 2/3

In [ ]:
import numpy as np
import rebound
import random
import timeit 

a_b, e_b = 1.,0.5
a_p, e_p = 4.,0.2    # a_p = 3 
mu = 0.5


In [ ]:

def setupSimulation(ab,eb,ap,ep,mu):
    sim = rebound.Simulation() #sim = rebound.Simulation()
    
    a_b,e_b = ab,eb
    m1 = 1.
    
    sim.add(m=m1, hash = "Star1") 
    
    m2 = (m1*mu)/(1-mu)
    inc_b = random.random()*np.pi
    Omega_b = random.random()*2*np.pi
    f_b=np.random.rand()*2.*np.pi
    sim.add(m =m2, a= a_b, e=e_b, inc =inc_b,Omega=Omega_b,f=f_b, hash = "Star2") 
    
    a_p,e_p = ap,ep
    inc_p = random.random()*np.pi
    Omega_p = random.random()*2*np.pi
    f_p=np.random.rand()*2.*np.pi
    sim.add(m=0.,a=a_p,e=e_p,inc=inc_p,Omega=Omega_p, f=f_p, hash = "Planet1")
    
    sim.move_to_com()
    return sim


In [ ]:
# import time
# import itertools

# start = time.time()
# num = 1
# for _ in itertools.repeat(None,num):
#     sim = setupSimulation(a_b,e_b,a_p,e_p,mu)
# end = time.time()

# print("Time elapsed:",end-start)
# sim.status()

In [ ]:
sim = setupSimulation(a_b,e_b,a_p,e_p,mu)
sim.exit_max_distance = 1000*a_b

Torb = 2.*np.pi
Noutputs = 1000
Norb_max = 1e4 #maximum number of orbital periods that will be used
Tmin = Torb 
Tmax = 30*Torb

times = np.linspace(Tmin, Tmax, Noutputs)
xS1,yS1 = np.zeros(Noutputs), np.zeros(Noutputs)
xS2,yS2 = np.zeros(Noutputs), np.zeros(Noutputs)
xP1,yP1 = np.zeros(Noutputs), np.zeros(Noutputs)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
for i,time in enumerate(times):
    try:
        sim.integrate(time)  
        print(error)
        for j in range(sim.N):
            p = sim.particles[j]
            d2 = p.x**2 + p.y**2 + p.z**2
            if d2>sim.exit_max_distance**2:
                sim.exit(1) # cache index rather than remove here since our loop would go beyond end of particles array
    xS1[i] = sim.particles["Star1"].x
    yS1[i] = sim.particles["Star1"].y
    xS2[i] = sim.particles["Star2"].x
    yS2[i] = sim.particles["Star2"].y
    xP1[i] = sim.particles["Planet1"].x
    yP1[i] = sim.particles["Planet1"].y

fig = plt.figure(figsize=(6,6), dpi= 100, facecolor='w', edgecolor='k')
ax = plt.subplot(121)
ax.set_xlim([-5,5])
ax.set_ylim([-5,5])


plt.plot(xS1,yS1, label = 'Star 1')
plt.plot(xS2,yS2, label = 'Star 2')
plt.plot(xP1,yP1, label = 'Test Particle 1 (planet)')
plt.xlabel("x")
plt.ylabel("y")
plt.title("Binary System Position Plot")
plt.legend()
plt.savefig('Sec2_Bin_Sim.png') 
#plt.savefig('S2_changea.png')


In [ ]:
%matplotlib inline
fig,ax = rebound.OrbitPlot(sim, color = True)
ax.set_title(" Binary System Position Plot(Using Orbitplot function) ")

plt.savefig('Sec2_OrbPlt.png')
#plt.savefig('OrbPlt_changea.png')


# CLASSIC RESULTS

In [ ]:
import numpy as np
import rebound
import random
from multiprocessing import Pool 

In [ ]:
# Calculates survival times for each tuple 
def Simulation(par):
    sim = rebound.Simulation()
    e_b,a_p = par #par[0],par[1]
    a_b = 1.
    m1 =1.
   
    sim.add(m=m1, hash = "Star1") 
    
    mu = 0.5
    m2 = (m1*mu)/(1-mu)
    inc_b = random.random()*np.pi
    Omega_b = random.random()*2*np.pi
    f_b=np.random.rand()*2.*np.pi
    sim.add(m =m2, a= a_b, e=e_b, inc =inc_b,Omega=Omega_b,f=f_b,  hash = "Star2")
    
    e_p = 0
    inc_p = random.random()*np.pi
    Omega_p = random.random()*2*np.pi
    f_p=np.random.rand()*2.*np.pi
    sim.add(m=0.,a=a_p,e=e_p,inc=inc_p,Omega=Omega_p, f=f_p, hash = "Planet1")
    
    #im.move_to_com()
    #sim.exit_max_distance = 1000*a_b
    max_dist = 100*a_b

    Torb = 2.*np.pi
    Noutputs = 100
    Norb_max = 1e4 
    Tmin = 0.
    Tmax = Norb_max*Torb
    times = np.linspace(Tmin, Tmax, Noutputs)
    #tmax = np.zeros(1)
    tmax_arr = []
    for i,time in enumerate(times):
        sim.integrate(time)
        p = sim.particles[2]
        d2 = p.x**2 + p.y**2 + p.z**2
        if(d2>max_dist**2):
            tmax_arr.append(time)
            #tmax = times[i]
            #print(tmax)
        
    return tmax_arr[0] #tmax    
    
        


In [ ]:
#Pool function to send an array of tuples (e_b and a_p values to Simulation() function)
random.seed(1)
N = 10
ab = 1
ebs = np.linspace(0.,0.7,N)
aps = ab*np.linspace(1.,5.,N)
params = [(eb,ap) for eb in ebs for ap in aps]

pool = rebound.InterruptiblePool()

# TO TIME ONE CALL TO THE 
#import time
#import itertools
#start = time.time()
#num = 1
#for _ in itertools.repeat(None,num):
#    stime = pool.map(Simulation,params) #survival times
#end = time.time()
#print("On a system with 2 cores and 4 logical processors, time elapsed is",end-start)

stime = pool.map(Simulation,params) #survival times
stime = np.array(stime).reshape(N,N)
stime = np.nan_to_num(stime)

#Colorplot of survival times
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.colors import LogNorm
import matplotlib

t,ax = plt.subplots(1,1,figsize=(7,5))
extent=[ebs.min(), ebs.max(), aps.min(), aps.max()]

ax.set_xlim(extent[0], extent[1])
ax.set_ylim(extent[2], extent[3])
ax.set_xlabel("Binary Eccentricity $e_b$ ")
ax.set_ylabel("Test particle semimajor axis $a_p$")
im = ax.imshow(stime, aspect='auto', origin="lower", interpolation='nearest', cmap="RdYlGn_r", extent=extent)

cb = plt.colorbar(im, ax=ax)
cb.solids.set_rasterized(True)
cb.set_label("Particle Survival Times")


In [ ]:
# e_b vs. a_b plot 
#import numpy as np
ebs = np.linspace(0.,0.7,N)
ab_s = np.zeros(N)
for i,eb in enumerate(ebs):
    ab_s[i] = 2.278 + 3.824*eb - 1.71*(eb**2)
#from matplotlib import pyplot as plt    
plt.plot(ebs,ab_s, marker = "^",markersize =5 )
plt.xlabel('$e_b$')
plt.ylabel('$a_b(a_c$)')
plt.title('Critical semimajor axis $a_c$ as a function of eccentricity $e_b$')
plt.savefig("ac_vs_eb.pdf")